<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Soft-Voting-Edited-Dataset/blob/main/Cross_Project_SV_train_maven%2Btest_greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
from numpy import mean
from numpy import std

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [116]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [117]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [118]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-maven 3.3.3-3.3.9 - maven 3.3.3-3.3.9.csv")


In [119]:
weights['Change'] = weights['Change'].map({'yes': 1, 'no': 0})

In [120]:
weights.shape

(831, 8)

In [121]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,22,10,26,237,1,73,13,0
1,2,0,18,40,0,40,5,0
2,4,0,24,71,0,27,11,0
3,2,2,16,15,0,0,3,0
4,3,0,14,7,0,0,1,0


In [122]:
X = weights.drop('Change', axis=1)
y = weights['Change']

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [124]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [125]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [126]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.7709090909090909, 0.7454545454545455, 0.6945454545454546]


In [127]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-greenstone 3.0.7 - greenstone 3.0.7.csv")

In [128]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [129]:
data.shape

(601, 8)

In [130]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,34,189,1,78,24,1
1,6,0,5,133,1,50,20,1
2,7,0,60,2124,1,0,355,1
3,21,0,15,145,1,85,17,1
4,10,0,13,721,1,91,27,1


In [131]:
X = data.drop('change', axis=1)
y = data['change']

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [133]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [134]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 85.950


In [135]:
print(yhat)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0]


In [136]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[101   1]
 [ 16   3]]


In [137]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = ensemble.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.7027863777089783


In [138]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.3013575551286118


In [139]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.574045407636739


In [140]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.39540706774918194


In [141]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92       102
           1       0.75      0.16      0.26        19

    accuracy                           0.86       121
   macro avg       0.81      0.57      0.59       121
weighted avg       0.85      0.86      0.82       121

